In [8]:
import sys
sys.path.insert(0, '/home/ma-user/work/pengjunran/workspace/GAIA_workspace/gaiavision/gaiavision')

import pandas as pd

from model_space.utils import fold_dict, unfold_dict
from model_space.model_samplers.builder import build_model_sampler
from model_space.model_samplers.random_model_sampler import RangeModelSampler, CandidateModelSampler
from model_space.model_samplers.functional_model_sampler import CompositeModelSampler, RepeatModelSampler, ConcatModelSampler, AnchorModelSampler
from model_space.model_space_manager import ModelSpaceManager

In [9]:
sw_ms = RangeModelSampler('arch.backbone.stem.width', 16, 32, 16, ascending=True)
bw_ms = RangeModelSampler('arch.backbone.body.width', [16,32,64], [32,64,128], [16,32,64], ascending=True)
bd_ms = RangeModelSampler('arch.backbone.body.depth', [2,4,2], [4,8,4], [1,2,1])
di_ms = CandidateModelSampler('data.input_shape', candidates=[480, 640])

ac_ms = AnchorModelSampler(anchors=[
    {'name': 'MAX', 'arch.backbone.stem.width': 32, 'arch.backbone.body.width': [32,64,128], 'arch.backbone.body.depth': [4,8,4]},
    {'name': 'MIN', 'arch.backbone.stem.width': 16, 'arch.backbone.body.width': [16,32,64], 'arch.backbone.body.depth': [2,4,2]}
])
cps_ms = CompositeModelSampler([di_ms, sw_ms, bw_ms, bd_ms])
rp_ms = RepeatModelSampler(cps_ms, times=3)
cc_ms = ConcatModelSampler([ac_ms, rp_ms])

In [ ]:
cc_ms.set_mode('traverse')
for mm in cc_ms.traverse():
    print(mm)

In [10]:
# archs = []
# for _ in range(len(cc_ms)):
#     archs.append(cc_ms.sample())
# print(archs)

[{'arch.backbone.stem.width': 32, 'arch.backbone.body.width': [32, 64, 128], 'arch.backbone.body.depth': [4, 8, 4]}, {'arch.backbone.stem.width': 16, 'arch.backbone.body.width': [16, 32, 64], 'arch.backbone.body.depth': [2, 4, 2]}, {'data.input_shape': 640, 'arch.backbone.stem.width': 32, 'arch.backbone.body.width': [16, 32, 128], 'arch.backbone.body.depth': [3, 8, 3]}, {'data.input_shape': 480, 'arch.backbone.stem.width': 32, 'arch.backbone.body.width': [32, 64, 128], 'arch.backbone.body.depth': [3, 8, 3]}, {'data.input_shape': 640, 'arch.backbone.stem.width': 32, 'arch.backbone.body.width': [16, 64, 128], 'arch.backbone.body.depth': [4, 6, 3]}]


In [4]:
ms = ModelSpaceManager.load(archs)
ms.head()

/home/ma-user/work/pengjunran/workspace/GAIA_workspace/gaiavision/gaiavision/model_space/model_space_manager.py:37: UserWarning: Lacking information of `performance`
  warnings.warn(f'Lacking information of `{k}`')
/home/ma-user/work/pengjunran/workspace/GAIA_workspace/gaiavision/gaiavision/model_space/model_space_manager.py:37: UserWarning: Lacking information of `metric`
  warnings.warn(f'Lacking information of `{k}`')
/home/ma-user/work/pengjunran/workspace/GAIA_workspace/gaiavision/gaiavision/model_space/model_space_manager.py:37: UserWarning: Lacking information of `data`
  warnings.warn(f'Lacking information of `{k}`')


,arch.backbone.stem.width,arch.backbone.body.width,arch.backbone.body.depth
0,32,"(32, 64, 128)","(4, 8, 4)"
1,16,"(16, 32, 64)","(2, 4, 2)"
2,32,"(16, 32, 128)","(2, 6, 4)"
3,32,"(16, 32, 128)","(4, 4, 2)"
4,32,"(32, 32, 64)","(2, 6, 3)"


In [13]:
s = set([True, False])
any(s)

True

In [5]:
cps_ms.set_mode('traverse')
d = cps_ms.sample()
fold_dict(d)

{'arch': {'backbone': {'body': {'depth': (4, 6, 4), 'width': (32, 32, 64)},
   'stem': {'width': 16}}}}

In [ ]:
list(cps_ms.traverse())

In [16]:
import json
from tqdm import tqdm
fin = open('/home/ma-user/work/pengjunran/workspace/GAIA_workspace/gaiadet/hubs/flops/old_ar50to101v2_flops.json', 'r')
fout = open('/home/ma-user/work/pengjunran/workspace/GAIA_workspace/gaiadet/hubs/flops/ar50to101v2_flops.json', 'w')
for l in tqdm(fin):
    d = json.loads(l.strip())
    input_shape = d['input_shape']
    overhead = {'flops':d['flops'], 'params': d['params']}
    sw = d['arch']['stem_width']
    bw = d['arch']['body_widths']
    bd = d['arch']['body_depths']
    arch = {'backbone':{'stem': {'width': sw}, 'body': {'width':bw, 'depth':bd}}}
    dout = {'data': {'input_shape': input_shape}, 'overhead': overhead, 'arch': arch}
    fout.write(json.dumps(dout)+'\n')
    

379080it [00:05, 67753.35it/s]


In [12]:
import sys
sys.path.insert(0, '/home/ma-user/work/pengjunran/workspace/GAIA_workspace/gaiavision')
sys.path.insert(0, '/home/ma-user/work/pengjunran/workspace/GAIA_workspace/gaiadet')
sys.path.insert(0, '/home/ma-user/work/pengjunran/workspace/GAIA_workspace/mmdetection')

from gaiadet.models import DynamicResNet

/home/ma-user/work/pengjunran/workspace/GAIA_workspace/gaiavision/gaiavision/model_space/model_space_manager.py:16: UserWarning: registration of accessor <class 'gaiavision.model_space.model_space_manager.ModelSpaceManager'> under name 'ms_manager' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  @pd.api.extensions.register_dataframe_accessor("ms_manager")


In [13]:
dr = DynamicResNet(in_channels=3, stem_width=64, body_width=[32,32,32,32], body_depth=[2,2,4,2])

NameError: name 'depth' is not defined

In [17]:
import pandas as pd
a = pd.DataFrame({'a': (1,2), 'b': (2,3)})

In [21]:
class A():
    def __setattr__(self, name, value):
        self.__dict__[name] = value
        print(f'{name}:{value}')


In [27]:
import torch
import torch.nn as nn

class A(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = nn.Linear(2,3)
        self.b = nn.Linear(3,4)
        
    def forward(self, x):
        return self.b(self.a(x))
    
a = A()
print(list(a.parameters()))
print(a._parameters)
print(a._modules)

[Parameter containing:
tensor([[0.3364, 0.6875],
        [0.6591, 0.5183],
        [0.1983, 0.0098]], requires_grad=True), Parameter containing:
tensor([ 0.7041, -0.3943,  0.4789], requires_grad=True), Parameter containing:
tensor([[-0.2075,  0.5752,  0.4051],
        [ 0.4945,  0.0029,  0.5199],
        [ 0.5592, -0.1572,  0.0059],
        [-0.0060,  0.0393,  0.4128]], requires_grad=True), Parameter containing:
tensor([-0.5284, -0.1626,  0.2612, -0.4031], requires_grad=True)]
OrderedDict()
OrderedDict([('a', Linear(in_features=2, out_features=3, bias=True)), ('b', Linear(in_features=3, out_features=4, bias=True))])


In [28]:
a = set([1,2])
a.